In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Pipeline, AutoModel
import transformers
import torch

In [85]:
x = torch.Tensor([
    [
        [1, 2, 3], 
        [4, 5, 6], 
        [7, 8, 9]
    ],
    [
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]
    ]
])

In [88]:
layer = torch.nn.Linear(3, 3)

In [94]:
res = torch.nn.functional.softmax(layer(x), dim=1)
res

tensor([[[0.8174, 0.1344, 0.1446],
         [0.1537, 0.2804, 0.2868],
         [0.0289, 0.5852, 0.5686]],

        [[0.8174, 0.1344, 0.1446],
         [0.1537, 0.2804, 0.2868],
         [0.0289, 0.5852, 0.5686]]], grad_fn=<SoftmaxBackward0>)

In [95]:
y = torch.Tensor([
    [
        [1, 0, 0],
        [0, 1, 0], 
        [0, 0, 1]
    ],
    [ 
        [1, 0, 0],
        [0, 1, 0], 
        [0, 0, 1]
    ]
])

In [96]:
loss = torch.nn.BCEWithLogitsLoss()

In [97]:
loss(res, y)

tensor(0.6959, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [2]:
    model = AutoModel.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain", local_files_only=True)
    
    print(model)
    tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain", local_files_only=True, truncation=True, padding=True)

    

Some weights of the model checkpoint at Hate-speech-CNERG/bert-base-uncased-hatexplain were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [45]:
pipe = transformers.pipeline(task='feature-extraction', model=model, tokenizer=tokenizer, truncation=True, padding=True, device=0, framework="pt")
    

In [3]:
res = tokenizer(["hello world my name is liam", "no way my name is also liam"], padding=True, truncation=True, return_tensors="pt")


In [75]:
res

{'input_ids': tensor([[ 101, 7592, 2088, 2026, 2171, 2003, 8230,  102,    0],
        [ 101, 2053, 2126, 2026, 2171, 2003, 2036, 8230,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [4]:
res2 = model.forward(**res)

In [6]:
res2.pooler_output.shape

torch.Size([2, 768])

In [41]:
res = pipe(["hello world my name is liam", "and my other name is also liam"])

In [42]:
res = torch.Tensor(res)[:, 0, :]

ValueError: expected sequence of length 8 at dim 2 (got 9)

In [40]:
res.shape

torch.Size([1, 768])

In [2]:
import dask.dataframe as dd
from dask.distributed import Client 

In [3]:

client = Client(n_workers=8)
client

/home/l2hebert/miniconda3/envs/reddit-graph/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44175 instead
  f"Port {expected} is already in use.\n"


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:44175/status,
Dashboard: http://127.0.0.1:44175/status,Workers: 8
Total threads: 24,Total memory: 125.83 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45085,Workers: 8
Dashboard: http://127.0.0.1:44175/status,Total threads: 24
Started: Just now,Total memory: 125.83 GiB
Comm: tcp://127.0.0.1:33347,Total threads: 3
Dashboard: http://127.0.0.1:34689/status,Memory: 15.73 GiB
Nanny: tcp://127.0.0.1:35263,


In [4]:
df = dd.read_parquet("~/reddit-processed/*")
df2 = dd.read_parquet("~/reddit-processed-other/*")
# df = df.loc[df['author'] != '[deleted]'] # We don't want this as it could break graph structure

In [5]:
df2.head(n=5)

,title,selftext,subreddit,score,created_utc,post_hint,secure_media,name,media_text,body,hate_score,normal_score,offensive_score
160,McConnell: Dropping liability protections from...,,politics,1,1596240009,None,None,t3_i1ih7z,,McConnell: Dropping liability protections from...,0.031498,0.789893,0.178610
515,I got told to draw something holy instead,,AmItheAsshole,1,1596240035,None,None,t3_i1ihij,,I got told to draw something holy instead\n,0.044338,0.764321,0.191341
544,Judge Will Decide Whether Postmark Problems Sh...,,politics,1,1596240037,None,None,t3_i1ihjc,,Judge Will Decide Whether Postmark Problems Sh...,0.068286,0.684849,0.246866
700,AITA for not telling my husband what I did whe...,,AmItheAsshole,2,1596240051,None,None,t3_i1ihnw,,AITA for not telling my husband what I did whe...,0.030186,0.724114,0.245700
1054,AITA for being furious with my mother that she...,,AmItheAsshole,1,1596240082,None,None,t3_i1ihyl,,AITA for being furious with my mother that she...,0.033410,0.729982,0.236607


In [6]:
df.columns
df = df.drop(['gilded', 'author_flair_text', 'author'], axis=1)

In [7]:
df2.columns
df2['parent_id'] = -1
df2['link_id'] = df2['name']
df2 = df2.rename(columns={'name': 'id'})
df2 = df2.drop(['post_hint', 'secure_media', 'media_text', 'title', 'selftext'], axis=1)

Index(['body', 'created_utc', 'score', 'parent_id', 'id', 'link_id',
       'subreddit', 'hate_score', 'normal_score', 'offensive_score'],
      dtype='object')

In [6]:
df2.sort_values(by=['offensive_score'], ascending=False).head(n=5)

,subreddit,score,created_utc,id,body,hate_score,normal_score,offensive_score,parent_id,link_id
16397651,AmItheAsshole,0,1597774439,t3_ic6d81,"AITA IM autistic, do I get a retard pass?\n",0.035981,0.130948,0.833070,-1,t3_ic6d81
12134597,AmItheAsshole,1,1597372125,t3_i9dnt1,Am I the asshole for using the word retard?\n,0.039281,0.172671,0.788048,-1,t3_i9dnt1
21261149,AmItheAsshole,0,1598218957,t3_ifbywo,AITA for *accidentally* spilling nachos on an ...,0.047630,0.168480,0.783890,-1,t3_ifbywo
2435838,AmItheAsshole,1,1596481760,t3_i33bwl,Am I the asshole? For thinking that trigger wa...,0.048972,0.167156,0.783872,-1,t3_i33bwl
12135355,AmItheAsshole,1,1597372195,t3_i9dofh,AITA for using the word retard?\n,0.038846,0.184073,0.777080,-1,t3_i9dofh


In [8]:
df = df.append(df2)

In [12]:
df['subreddit'].value_counts().compute()

The_Donald         6531455
politics           6127723
AmItheAsshole      4135270
IAmA                183284
climateskeptics      27382
Name: subreddit, dtype: int64

In [13]:
import pandas as pd
import numpy as np
child_count = df.groupby(by=['link_id']).count()[['body']]

merged = df.merge(child_count, on='link_id', how='left')
merged = merged.rename(columns={'body_y': 'total_conversation_size', 'body_x': 'body'})
merged = merged.loc[merged['total_conversation_size'] > 3]

In [15]:
merged['subreddit'].value_counts().compute()

The_Donald         6058099
politics           6002954
AmItheAsshole      4013869
IAmA                178297
climateskeptics      25124
Name: subreddit, dtype: int64

In [38]:
#merged[['body', 'id']].to_parquet('/home/l2hebert/reddit-text') # we will process this on the fly
merged = merged.drop('body', axis=1)


In [31]:
merged.head(n=10)

KeyboardInterrupt: 

In [39]:
import torch
from torch_geometric.data import Data
merged['parent_id'] = merged['parent_id'].apply(lambda x: x[3:], meta=str) 
merged['link_id'] = merged['link_id'].apply(lambda x: x[3:], meta=str) 
merged['comp_score'] = ((merged['offensive_score'] + merged['hate_score']) * 2) - 1 # goes to -1 to 1
merged['created_utc'] = ((merged['created_utc'])) / 86400
merged['created_utc'] = merged['created_utc'].astype(int)

merged = merged.compute()




In [40]:
merged['created_utc'].value_counts()

18249    225612
18248    195152
18075    170758
18250    163442
18242    163105
          ...  
16631         1
16634         1
16623         1
16638         1
16650         1
Name: created_utc, Length: 523, dtype: int64

In [46]:
client.close()

In [45]:
grouped = merged.groupby(by=['link_id'])
print('total groups', len(grouped))
import os 

def compute_weighted_scores(x, mapping):
    leafs = x.loc[(~x['index'].isin(x['parent_id']))]
    leafs = leafs.loc[leafs['parent_id'] != -1]['index']
    ids = list(leafs.index)
    leafs = list(leafs)
    while len(leafs) != 0:
        curr = leafs.pop(0)
        curr_index = ids.pop(0)
        curr_data = x.loc[x['index'] == curr]
        if len(curr_data) != 1:
            print('CURR DATA LENGTH')
            print(curr_data)
            print(x)

        children = x.loc[x['parent_id'] == curr]
        
        if len(children) != 0:
            children = list(0.25 * children['weighted_score'])
        else:
            children = []
        
        #print(float(curr_data['score']), float(curr_data['upvotes']), float(parent['score']), float(parent['upvotes']))
    
        new_score = 0
        for val in children:
            new_score += val

        if int(curr_data['parent_id']) != -1:
            parent = x.loc[x['index'] == int(curr_data['parent_id'])]
            if len(parent) != 1:
                print("PARENT DATA LENGTH")
                print(parent)
                print("MAPPING")
                print(mapping)
                print("LOOKING FOR ", curr_data['parent_id'])
                print(x)
                raise Exception
            new_score += 0.25 * float(parent['comp_score']) * float(parent['score'])
            new_score += 0.75 * float(curr_data['comp_score']) * float(curr_data['score']) # worth 3x as much
            x.at[curr_index, 'weighted_score'] = new_score
            parent_id = int(parent['index'])
            if not parent_id in leafs:
                leafs.append(parent_id)
                ids.append(int(parent.index[0]))
        else:
            new_score += 0.75 * float(curr_data['comp_score']) * float(curr_data['score'])
            x.at[curr_index, 'weighted_score'] = new_score
    
    # multiple -1 should all influence each other as a proxy for no head comment
    head_nodes = x.loc[x['parent_id'] == -1]
    if len(head_nodes) != 1:
        ids = list(head_nodes.index)
        if 'weighted_score' not in head_nodes.columns:
            head_nodes['temp_weighted_score'] = head_nodes['score'] * head_nodes['comp_score'] * 0.75
        else:
            head_nodes['temp_weighted_score'] = head_nodes['weighted_score']
        # leafs = list(head_nodes['index'])
        
        for id in ids:
            curr = head_nodes.loc[id]
            rest = list(head_nodes.loc[head_nodes['index'] != curr['index']]['temp_weighted_score'])
            new_score = float(curr['temp_weighted_score'])
            for score in rest:
                new_score += 0.25 * score
            
            x.at[id, 'weighted_score'] = new_score
    
    # Convert scores to catagories [normal, 0-5, 5-20, 20 - 500, 500+]

def convert_to_catagories(x: pd.Series):
    normal = [1 if k <= 0 else 0 for k in x]
    small = [1 if 0 < k < 5 else 0 for k in x]
    mid = [1 if 5 <= k < 20 else 0 for k in x]
    large = [1 if 20 <= k < 500 else 0 for k in x]
    massive = [1 if 500 <= k else 0 for k in x]

    return pd.DataFrame({'normal': normal, 'small': small, 'mid':mid, 'large':large, 'massive':massive})

def grouped_apply(x, name):
    x['index'] = 1 
    x['index'] = x['index'].cumsum()
    x['index'] = x['index'] - 1 # so we start from 0
    mapping = x[['index', 'id']].set_index('id').to_dict('index')
    mapping = {id: value['index'] for id, value in mapping.items()}
    mapping['-1'] = -1
    mapping[name] = -1

    x = x.replace({'parent_id': mapping})
    
    #x['parent_id'] = pd.to_numeric(x['parent_id'], errors='coerce', downcast='integer').fillna(-1) # replace all missing parents as if the nodes pointing to them are the head. It's not perfect
    x['index'] = pd.to_numeric(x['index']) # to fix floating point bs
    
    x['parent_id'] = x['parent_id'].astype(np.int64)
    # x['index'] = x['index'].astype(np.int64)
    if len(x['index'].unique()) != len(x):
        print('INDEX LENGTH')
        print(len(x['index'].unique()), ' vs ', len(x))
        print(x)

    compute_weighted_scores(x, mapping)
    if 'weighted_score' not in x.columns:
        print('MISSING SCORES')
        print(x)
        return
    else:
        if len(x.loc[x['weighted_score'] == np.nan]) != 0:
            print('NAN LENGTH')
            print(x)
            return

    data = torch.Tensor(x[['score']].values) # text data eventually appended at run time     
    
    edge_matrix = torch.Tensor([[], []])
    
    
    for i, row in x.iterrows():
        if row['parent_id'] == -1:  
            edges = torch.Tensor([[row['index']], [row['index']]])
        else:
            edges = torch.Tensor([[row['index'], row['index'], row['parent_id']], [row['parent_id'], row['index'], row['index']]])
        
        edge_matrix = torch.cat((edge_matrix, edges), dim=1) 
    #print(edge_matrix)
    
    edge_features = torch.zeros((edge_matrix.shape[1], 0))

    x[['normal', 'small', 'mid', 'large', 'massive']] = convert_to_catagories(x['weighted_score'])
    data = Data(data, edge_matrix, edge_features, torch.Tensor(x[['normal', 'small', 'mid', 'large', 'massive']].values))
    
    subreddit = str(x['subreddit'].iloc[0])
    created_utc = str(int((x['created_utc'].sum())/len(x)))
    os.makedirs('/home/l2hebert/reddit-graph/' + subreddit + '/' + created_utc, exist_ok=True)
    torch.save(data, '/home/l2hebert/reddit-graph/' + subreddit + '/' + created_utc + '/' + name + '.pt')
    #print(data)

    # need a data matrix (rows, data)
    # - matter of picking what exact rows we want, and then going from there
    # need an edge matrix (2, edges) COO Format??
    # - COO format is just index - index format
    # - so, this should just be node idx - parent idx
    # - this should be a groupby inside the groupby, set everything to the id to 1? 
    # - another idea would be to set an id to everything (cumsum) and then set 1 for each direction?
    # need an edge attr (num_edges, num_features) I dont think we have this
    # Need graph-level or node-level labels (graph level?)

grouped.apply(lambda x: grouped_apply(x, x.name)).compute()




total groups 333487


KeyboardInterrupt: 

In [12]:
len(df)

5158645

In [19]:
data = {
    'a': [i for i in range(3)],
    'b': ['a', 2, 'c']
}
test = pd.DataFrame(data)
test

,a,b
0,0,a
1,1,2
2,2,c


In [22]:
test['b'] = pd.to_numeric(test['b'], errors='coerce').fillna(-1)

In [23]:
test

,a,b
0,0,-1.0
1,1,2.0
2,2,-1.0


In [2]:
from math import isclose
import dask.dataframe as dd
from dask.distributed import Client
from numpy.lib.index_tricks import index_exp 
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from tqdm import tqdm
import os 
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Pipeline, AutoModel
import transformers


In [12]:
client = Client(n_workers=8)


In [13]:

# model = AutoModel.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain", local_files_only=True)

# tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain", local_files_only=True)

df = dd.read_parquet("~/reddit-processed/*")
df2 = dd.read_parquet("~/reddit-processed-other/*")
# df = df.loc[df['author'] != '[deleted]'] # We don't want this as it could break graph structure

df = df.drop(['gilded', 'author_flair_text', 'author'], axis=1)

df2['parent_id'] = -1
df2['link_id'] = df2['name']
df2 = df2.rename(columns={'name': 'id'})
df2 = df2.drop(['post_hint', 'secure_media', 'media_text', 'title', 'selftext'], axis=1)

df = df.append(df2)


child_count = df.groupby(by=['link_id']).count()[['body']]

merged = df.merge(child_count, on='link_id', how='left')
merged = merged.rename(columns={'body_y': 'total_conversation_size', 'body_x': 'body'})
merged = merged.loc[merged['total_conversation_size'] > 3]

merged['parent_id'] = merged['parent_id'].apply(lambda x: x[3:], meta=str) 
merged['link_id'] = merged['link_id'].apply(lambda x: x[3:], meta=str) 
merged['comp_score'] = ((merged['offensive_score'] + merged['hate_score'] - 0.35) * 2) # goes to -0.7 to 1.5
merged['created_utc'] = ((merged['created_utc'])) / 86400
merged['created_utc'] = merged['created_utc'].astype(int)

grouped = merged.groupby(by=['link_id']).mean()[['created_utc']].astype(int)
merged = merged.merge(grouped, on='link_id', how='left')
merged = merged.rename(columns={'created_utc_y': 'mean_utc', 'created_utc_x': 'created_utc'})

# merged['']
# data_to_write = merged[['body', 'link_id']]
# merged[['body', 'link_id']].to_parquet('/home/l2hebert/reddit-text') # we will process this on the fly
# merged = merged.drop('body', axis=1)



# def tokenized(x):
#     data = list(x['body'])
#     if len(data) == 0:
#         x['embedding'] = []
#     else:
#         res = tokenizer(list(x['body']), padding='max_length', truncation=True, return_tensors='pt')
#         with torch.no_grad():
#             res = model(**res)
            
#             x['embedding'] = res.pooler_output.tolist()
        
#     return x

# merged = merged.map_partitions(tokenized)
merged['index'] = 1 

# grouped = merged.groupby(by=['link_id']).mean()[['created_utc']].astype(int)
# merged = merged.merge(grouped, on='link_id', how='left')
# merged = merged.rename(columns={'created_utc_y': 'mean_utc', 'created_utc_x': 'created_utc'})



#merged['index'] = merged.groupby(by=['link_id']).cumsum(axis=1)['index']
merged['path'] = '/home/l2hebert/reddit-graph/' + merged['subreddit'] + '/' + merged['mean_utc'].astype(str) + '/' + merged['link_id'] + '.pt'

merged = merged.persist()
#merged[['body', 'path', 'link_id', 'index']].to_parquet('/home/l2hebert/reddit-text2', schema="infer") # lets pray its the same order
merged = merged.drop(['body'], axis=1)
merged = merged.compute()

client.close()

grouped = merged.groupby(by=['link_id'])



    #print(data)

    # need a data matrix (rows, data)
    # - matter of picking what exact rows we want, and then going from there
    # need an edge matrix (2, edges) COO Format??
    # - COO format is just index - index format
    # - so, this should just be node idx - parent idx
    # - this should be a groupby inside the groupby, set everything to the id to 1? 
    # - another idea would be to set an id to everything (cumsum) and then set 1 for each direction?
    # need an edge attr (num_edges, num_features) I dont think we have this
    # Need graph-level or node-level labels (graph level?)


In [14]:
from tqdm.notebook import tqdm

total groups 333487


  0%|          | 0/333487 [00:00<?, ?it/s]

  0%|          | 0/333487 [01:14<?, ?it/s]


In [28]:
def compute_weighted_scores(x, mapping):
    leafs = x.loc[(~x['index'].isin(x['parent_id']))]
    leafs = leafs.loc[leafs['parent_id'] != -1]['index']
    ids = list(leafs.index)
    leafs = list(leafs)

    while len(leafs) != 0:
        curr = leafs.pop(0)
        curr_index = ids.pop(0)
        curr_data = x.loc[x['index'] == curr]
        if len(curr_data) != 1:
            print('CURR DATA LENGTH')
            print(curr_data)
            print(x)

        children = x.loc[x['parent_id'] == curr]
        
        if len(children) != 0:
            subtree_length = children['subtree_length'].max() + 1
            children = list(0.25 * children['weighted_score'])
            
        else:
            subtree_length = 0
            new_score = 0
            children = []
        
        #print(float(curr_data['score']), float(curr_data['upvotes']), float(parent['score']), float(parent['upvotes']))
    
        new_score = 0
        for val in children:
            new_score += val

        if int(curr_data['parent_id']) != -1:
            parent = x.loc[x['index'] == int(curr_data['parent_id'])]
            if len(parent) != 1:
                print("PARENT DATA LENGTH")
                print(parent)
                print("MAPPING")
                print(mapping)
                print("LOOKING FOR ", curr_data['parent_id'])
                print(x)
                raise Exception
            new_score += 0.25 * float(parent['comp_score']) * float(parent['score'])
            new_score += 0.75 * float(curr_data['comp_score']) * float(curr_data['score']) # worth 3x as much
            
            parent_id = int(parent['index'])
            if not parent_id in leafs:
                leafs.append(parent_id)
                ids.append(int(parent.index[0]))
        else:
            new_score += 0.75 * float(curr_data['comp_score']) * float(curr_data['score'])
        
        x.at[curr_index, 'weighted_score'] = new_score
        x.at[curr_index, 'subtree_length'] = subtree_length
    
    # multiple -1 should all influence each other as a proxy for no head comment
    head_nodes = x.loc[x['parent_id'] == -1]
    if len(head_nodes) != 1:
        ids = list(head_nodes['index'])
        temp_scores = []
        subtree_length = []
        for id in ids:
            curr = head_nodes.loc[head_nodes['index'] == id]
          
            if 'weighted_score' not in curr or pd.isna(curr['weighted_score'].iloc[0]):
                temp_scores += [curr['score'].iloc[0] * curr['comp_score'].iloc[0] * 0.75]
            else:
                temp_scores += [curr['weighted_score'].iloc[0]]
            
            if 'subtree_length' not in curr or pd.isna(curr['subtree_length'].iloc[0]):
                subtree_length += [0]
            else:
                subtree_length += [curr['subtree_length'].iloc[0]] 
            # if 'weighted_score' not in head_nodes.columns: # also means no children nodes, just a bunch of stranglers
            #     #head_nodes['subtree_length'] = 0
            #     x.loc[x['parent_id'] == -1, 'subtree_length'] = 0
            #     head_nodes['temp_weighted_score'] = head_nodes['score'] * head_nodes['comp_score'] * 0.75
            # else:
            #     head_nodes['temp_weighted_score'] = head_nodes['weighted_score']
        head_nodes['temp_weighted_score'] = temp_scores 
        head_nodes['subtree_length'] = subtree_length
        # leafs = list(head_nodes['index'])
        
        for id in ids:
            curr = head_nodes.loc[head_nodes['index'] == id]
            score = float(curr['temp_weighted_score'])
            subtree_length = int(curr['subtree_length'])
            curr_index = int(curr['index'])
            if type(curr) == pd.Series:
                print("type curr error")
                print(curr)
                print('doing workaround, not ideal')
                rest = head_nodes
                curr = curr.iloc[0]
                new_score = curr['temp_weighted_score']
            else:
                rest = list(head_nodes.loc[head_nodes['index'] != curr_index]['temp_weighted_score'])
                new_score = float(score)
            for score in rest:
                new_score += 0.25 * score
                
            
            x.loc[x['index'] == curr_index, 'weighted_score'] = new_score
            x.loc[x['index'] == curr_index, 'subtree_length'] = subtree_length
        
        
    
    # Convert scores to catagories [normal, 0-5, 5-20, 20 - 500, 500+]

def convert_to_catagories(x: pd.Series):
    normal = [1 if k <= 0 else 0 for k in x]
    small = [1 if 0 < k < 5 else 0 for k in x]
    mid = [1 if 5 <= k < 20 else 0 for k in x]
    large = [1 if 20 <= k < 500 else 0 for k in x]
    massive = [1 if 500 <= k else 0 for k in x]

    return normal, small, mid, large, massive

global counter 
counter = 0
global culled 
culled = 0


def grouped_apply(x, name):
    global counter
    global culled
    counter += 1
    if counter % 10 == 0:
        t.set_postfix({'processing': name, 'culled': culled}, refresh=False)
        t.update(10)
    # if counter < 202750:
    #     subreddit = str(x['subreddit'].iloc[0])
    #     created_utc = str(int((x['created_utc'].sum())/len(x)))
    #     x['path'] = '/home/l2hebert/reddit-graph/' + subreddit + '/' + created_utc + '/' + name + '.pt'
    #     return x

    x['index'] = 1 
    x['index'] = x['index'].cumsum()
    x['index'] = x['index'] - 1 # so we start from 0
    mapping = x[['index', 'id']].set_index('id').to_dict('index')
    mapping = {id: value['index'] for id, value in mapping.items()}
    mapping['-1'] = -1
    mapping[name] = -1

    x = x.replace({'parent_id': mapping})
    x['parent_id'] = x['parent_id'].apply(lambda x: x if type(x) != str else -1)
    
    #x['parent_id'] = pd.to_numeric(x['parent_id'], errors='coerce', downcast='integer').fillna(-1) # replace all missing parents as if the nodes pointing to them are the head. It's not perfect
    x['index'] = pd.to_numeric(x['index']) # to fix floating point bs
    
    x['parent_id'] = x['parent_id'].astype(np.int64)
    # x['index'] = x['index'].astype(np.int64)
    if len(x['index'].unique()) != len(x):
        print('INDEX LENGTH')
        print(len(x['index'].unique()), ' vs ', len(x))
        print(x)

    compute_weighted_scores(x, mapping)

    #largest_length = x['subtree_length'].max()
    before = len(x)
    # print(x['subtree_length'])
    x = x.loc[x['subtree_length'] > 1]
    after = len(x)
    culled += before - after
    # print('sizes', before, after)
    # print(x['subtree_length'])
    
    if 'weighted_score' not in x.columns:
        print('MISSING SCORES')
        print(x)
        return
    else:
        if len(x.loc[x['weighted_score'] == np.nan]) != 0:
            print('NAN LENGTH')
            print(x)
            return

    data = torch.Tensor(x[['score']].values) # text data eventually appended at run time     
    
    edge_matrix = torch.Tensor([[], []])
    
    
    for i, row in x.iterrows():
        if row['parent_id'] == -1:  
            edges = torch.Tensor([[row['index']], [row['index']]])
        else:
            edges = torch.Tensor([[row['index'], row['index'], row['parent_id']], [row['parent_id'], row['index'], row['index']]])
        
        edge_matrix = torch.cat((edge_matrix, edges), dim=1) 
    #print(edge_matrix)
    
    edge_features = torch.zeros((edge_matrix.shape[1], 0))

    normal, small, mid, large, massive = convert_to_catagories(x['weighted_score'])
    x['normal'] = normal 
    x['small'] = small
    x['mid'] = mid 
    x['large'] = large 
    x['massive'] = massive
    
    #data = Data(data, edge_matrix, edge_features, torch.Tensor(x[['normal', 'small', 'mid', 'large', 'massive']].values))
    
    #data.index = x['index']
    #os.makedirs('/home/l2hebert/reddit-graph/' + str(x['subreddit'].iloc[0]) + '/' + str(x['created_utc'].iloc[0]), exist_ok=True)
    #x['path'] = '/home/l2hebert/reddit-graph/' + subreddit + '/' + created_utc + '/' + name + '.pt'
    #torch.save(data, str(x['path'].iloc[0]))
    
    return x

In [29]:
print('total groups', grouped.ngroups)
t = tqdm(total=grouped.ngroups)


total groups 333487


  0%|          | 0/333487 [00:00<?, ?it/s]

In [30]:
test = grouped_apply(grouped.get_group('bs9yau'), 'bs9yau')
test

# merged = grouped.apply(lambda k: grouped_apply(k, k.name))
# print('done merged')
# t.close()
# merged[['index', 'body', 'path']].to_parquet('/home/l2hebert/reddit-text/data.parquet')
# print('DONE!')

/home/l2hebert/miniconda3/envs/reddit-graph/lib/python3.7/site-packages/ipykernel_launcher.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/l2hebert/miniconda3/envs/reddit-graph/lib/python3.7/site-packages/ipykernel_launcher.py:139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/l2hebert/miniconda3/envs/reddit-graph/lib/python3.7/site-packages/ipykernel_launcher.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

10144    0.0
10243    0.0
13310    0.0
4302     0.0
10298    1.0
        ... 
20490    0.0
2744     0.0
15711    0.0
14960    0.0
3785     0.0
Name: subtree_length, Length: 152, dtype: float64
sizes 152 14
23081    2.0
12634    2.0
10477    3.0
10704    2.0
16207    2.0
13690    5.0
20204    4.0
20419    3.0
20507    2.0
6719     2.0
16304    2.0
10088    2.0
4065     2.0
148      2.0
Name: subtree_length, dtype: float64


,created_utc,score,parent_id,id,link_id,subreddit,hate_score,normal_score,offensive_score,total_conversation_size,...,mean_utc,index,path,weighted_score,subtree_length,normal,small,mid,large,massive
23081,18051,2,-1,epzuimk,bs9yau,AmItheAsshole,0.052020,0.732884,0.215096,152,...,18060,28,/home/l2hebert/reddit-graph/AmItheAsshole/1806...,-1.970480,2.0,1,0,0,0,0
12634,18054,1,-1,eq9v5fm,bs9yau,AmItheAsshole,0.027510,0.824100,0.148390,152,...,18060,43,/home/l2hebert/reddit-graph/AmItheAsshole/1806...,-1.953576,2.0,1,0,0,0,0
10477,18056,2,-1,eqjeufw,bs9yau,AmItheAsshole,0.038465,0.736794,0.224741,152,...,18060,55,/home/l2hebert/reddit-graph/AmItheAsshole/1806...,-1.885503,3.0,1,0,0,0,0
10704,18056,2,55,eqjfa38,bs9yau,AmItheAsshole,0.046034,0.630209,0.323757,152,...,18060,56,/home/l2hebert/reddit-graph/AmItheAsshole/1806...,-0.081446,2.0,1,0,0,0,0
16207,18057,3,-1,eqldema,bs9yau,AmItheAsshole,0.036624,0.751875,0.211501,152,...,18060,63,/home/l2hebert/reddit-graph/AmItheAsshole/1806...,-2.088548,2.0,1,0,0,0,0
13690,18059,1,-1,eqyfpyj,bs9yau,AmItheAsshole,0.040651,0.573059,0.386290,152,...,18060,69,/home/l2hebert/reddit-graph/AmItheAsshole/1806...,-1.647129,5.0,1,0,0,0,0
20204,18060,3,69,eqyqbb3,bs9yau,AmItheAsshole,0.052007,0.721771,0.226222,152,...,18060,71,/home/l2hebert/reddit-graph/AmItheAsshole/1806...,-0.313294,4.0,1,0,0,0,0
20419,18060,1,71,eqyqon7,bs9yau,AmItheAsshole,0.061224,0.612613,0.326163,152,...,18060,72,/home/l2hebert/reddit-graph/AmItheAsshole/1806...,-0.115175,3.0,1,0,0,0,0
20507,18060,2,72,eqyquht,bs9yau,AmItheAsshole,0.037874,0.722991,0.239135,152,...,18060,73,/home/l2hebert/reddit-graph/AmItheAsshole/1806...,-0.254393,2.0,1,0,0,0,0
6719,18063,1,-1,eraum2x,bs9yau,AmItheAsshole,0.027949,0.614413,0.357637,152,...,18060,96,/home/l2hebert/reddit-graph/AmItheAsshole/1806...,-1.687678,2.0,1,0,0,0,0
